In [1]:
import utils.rfi.sat_sim.satellite_simulations as ss
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import h5py

In [3]:
import dask

In [4]:
import dask.array as da

In [5]:
from dask import delayed

In [7]:
s0 = time.time()

In [8]:
@delayed
def RIME(B, K, E, G, autos=False):
    """
    Calculate the RIME for a given brightness matrix, 
    phase delay matrix, direction-dependent and 
    direction-independent effects. 
    
    Parameters:
    -----------
    B: np.array (2,2,n_time,n_freq,n_src)
        The brightness tensor.
    K: np.array (n_time,n_freq,n_ant,n_src)
        The phase delay tensor.
    E: np.array (2,2,n_time,n_freq,n_ant,n_src)
        The direction dependent effects tensor.
    G: np.array (2,2,n_time,n_freq,n_ant)
        The direction independent effects.
        
    Returns:
    --------
    V: np.array (2,2,n_time,n_freq,n_bl)
        The visibilies tensor. n_bl = n_ant*(n_ant-1)/2
    """
    
    a1, a2 = np.triu_indices(G.shape[-1], 0 if autos else 1)
    
#     Source coherency tensor
    X = B[:,:,:,:,None,:]*K[None,None,:,:,a1,:]*np.conjugate(K[None,None,:,:,a2,:])
    
#     Apparent source coherency
    A = np.einsum(('iAtfbs,ABtfbs,jBtfbs->ijtfbs'), E[...,a1,:], X, 
                  np.conjugate(E[...,a2,:]), optimize='optimal')
    
#     Visibilities
    V = np.einsum('iAtfb,ABtfbs,jBtfb->ijtfb', G[...,a1], A, 
                  np.conjugate(G[...,a2]), optimize='optimal')
    
    return V

# Get Brightness matrix

In [13]:
n_time, n_freq, n_ant, n_rfi = [800, 4096, 64, 6]

In [14]:
B = da.from_npy_stack('brightness_matrix/')

# Calculate phase delays

In [16]:
K = da.from_npy_stack('phase_delays/')

In [17]:
K

,Array,Chunk
Bytes,10.07 GB,2.46 MB
Shape,"(800, 4096, 64, 6)","(800, 1, 64, 6)"
Count,4096 Tasks,4096 Chunks
Type,complex64,numpy.ndarray


# Calculate direction dependent effects

In [18]:
E = da.from_npy_stack('DDEs/')

In [19]:
E

,Array,Chunk
Bytes,40.27 GB,9.83 MB
Shape,"(2, 2, 800, 4096, 64, 6)","(2, 2, 800, 1, 64, 6)"
Count,4096 Tasks,4096 Chunks
Type,complex64,numpy.ndarray


# Calculate direction independent effects

In [20]:
G = da.from_npy_stack('DIEs/')

In [21]:
G

,Array,Chunk
Bytes,6.71 GB,1.64 MB
Shape,"(2, 2, 800, 4096, 64)","(2, 2, 800, 1, 64)"
Count,4096 Tasks,4096 Chunks
Type,complex64,numpy.ndarray


# Calculate visibilities

In [22]:
s = time.time()

In [23]:
# n_time = 400

In [24]:
# %%time
Vs = []
for i in range(n_freq):
    V = RIME(B[:,:,:,i,None], K[:,i,None], E[:,:,:,i,None], G[:,:,:,i,None])
    Vs.append(V)

In [30]:
V = [da.from_delayed(Vs[i], (2,2,n_time,1,n_ant*(n_ant-1)//2), dtype=np.complex64) for i in range(n_freq)]
V = da.concatenate(V, axis=3).rechunk({3: 2})

In [31]:
V

,Array,Chunk
Bytes,211.39 GB,103.22 MB
Shape,"(2, 2, 800, 4096, 2016)","(2, 2, 800, 2, 2016)"
Count,63488 Tasks,2048 Chunks
Type,complex64,numpy.ndarray


In [39]:
s = time.time()

In [40]:
da.to_npy_stack('Vis/', V, axis=3)

In [41]:
print(time.time()-s)

2376.496368408203


In [32]:
s = time.time()

In [33]:
da.to_zarr(V, 'vis.zarr', overwrite=True)

In [34]:
print(time.time()-s)

2408.3559653759003


In [29]:
2562/60

42.7

In [ ]:
# fp = h5py.File('vis.h5', 'r')
# vis = fp['/rfi_vis']
# V = da.from_array(vis)

In [ ]:
# plt.plot(freqs/1e6, 10*np.log10(np.abs(V[0,0,0,:,0])))

In [ ]:
print(time.time()-s0)

In [23]:
da.to_zarr(V, 'vis.zarr', overwrite=True, compute=False).visualize('graph.pdf')

In [ ]:
# V.to_hdf5('vis.h5', '/rfi_vis', compute=False).visualize("graph.pdf")